In [1]:
##library importing
import pandas as pd
import numpy as np
import sklearn as sk
import datetime as dt
import dateutil.easter as ea
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import CCA
import xgboost as xgb





ModuleNotFoundError: No module named 'xgboost'

In [2]:
##data importing and type casting

forecastdemand_nsw = pd.read_csv(r'C:\Users\Aaron\Documents\Academic\Final project\Data\forecastdemand_nsw.csv')

temperature_nsw = pd.read_csv(r'C:\Users\Aaron\Documents\Academic\Final project\Data\temperature_nsw.csv')

truedemand_nsw = pd.read_csv(r'C:\Users\Aaron\Documents\Academic\Final project\Data\totaldemand_nsw.csv')



In [6]:
## now have all data in respective data frames
## alter datetimes fields to datetime data type

forecastdemand_nsw['DATETIME'] = forecastdemand_nsw['DATETIME'].apply(lambda x: pd.to_datetime(x))
forecastdemand_nsw['LASTCHANGED'] = forecastdemand_nsw['LASTCHANGED'].apply(lambda x: pd.to_datetime(x))

temperature_nsw['DATETIME'] = temperature_nsw['DATETIME'].apply(lambda x: pd.to_datetime(x))
##temperature_nsw['ADJUSTED_DATETIME'] = temperature_nsw['DATETIME'].apply(lambda x: pd.datetime(x.year
##                                                                                               ,x.month
##                                                                                               ,x.day
##                                                                                               ,x.hour
##                                                                                               ,0
##                                                                                               ,x.second) 
##                                                                                               + np.timedelta64(30*round(x.minute/30),'m'))
temperature_nsw['MINUTE'] = temperature_nsw['DATETIME'].apply(lambda x:x.minute)

truedemand_nsw['DATETIME'] = truedemand_nsw['DATETIME'].apply(lambda x: pd.to_datetime(x))

##prep temperature data for interpolating by removing erroneous data points (e.g. one temp was -9999)
temperature_nsw = temperature_nsw[temperature_nsw['TEMPERATURE']>-30]

In [29]:
P1_forecast_demand = pd.DataFrame(forecastdemand_nsw[forecastdemand_nsw['PERIODID'] ==1],columns=forecastdemand_nsw.columns)
P1_forecast_demand['DATEINT'] = forecastdemand_nsw['DATETIME'].astype(np.int64)
P1_true_demand = pd.DataFrame(truedemand_nsw[['DATETIME','TOTALDEMAND']],columns = truedemand_nsw.columns)
P1_true_demand['DATEINT'] = truedemand_nsw['DATETIME'].astype(np.int64)
                                                                                    
print(P1_true_demand.dtypes)
print(P1_forecast_demand.dtypes)
comp_set = P1_forecast_demand.join(P1_true_demand,how = 'inner',lsuffix = '_X',rsuffix = '_Y')
mean_squared_error(comp_set['TOTALDEMAND'],comp_set['FORECASTDEMAND'])

##print(comp_set)

DATETIME       datetime64[ns]
REGIONID              float64
TOTALDEMAND           float64
DATEINT                 int64
dtype: object
DATETIME            datetime64[ns]
REGIONID                    object
FORECASTDEMAND             float64
PREDISPATCHSEQNO             int64
PERIODID                     int64
LASTCHANGED         datetime64[ns]
DATEINT                      int64
dtype: object
                 DATETIME_X REGIONID_X  FORECASTDEMAND  PREDISPATCHSEQNO  \
68      2010-01-01 00:00:00       NSW1         7999.11        2009123140   
138     2010-01-01 00:30:00       NSW1         7596.21        2009123141   
209     2010-01-01 01:00:00       NSW1         7380.70        2009123142   
281     2010-01-01 01:30:00       NSW1         7022.05        2009123143   
354     2010-01-01 02:00:00       NSW1         6682.92        2009123144   
...                     ...        ...             ...               ...   
1323119 2011-05-12 17:30:00       NSW1        10701.35        2011051227   

In [23]:
##public holiday determination

def public_holiday_check(new_date_check: str):
    new_date = pd.Timestamp(new_date_check)
    ##new year and australia day
    if new_date.month == 1:
        if new_date.weekday() not in (5,6):
            if new_date.day in (1,26):
                return(1)
        elif new_date.weekday() == 5:
            if new_date.day in (3,28):
                return(1)
        else:
            if new_date.day in (2,27):
                return(1)
    if new_date.month == 3 & ea.easter(new_date.year).month ==3:
        if new_date.day in range(ea.easter(new_date.year).day-2,ea.easter(new_date.year).day+1):
            return(1)
    ##Anzac day
    if new_date.month == 4:
        if new_date.weekday() not in (5,6):
            if new_date.day == 25:
                return(1)
        elif new_date.weekday() == 6:
            if new_date.day == 26:
                return(1)
        else:
            if new_date.day == 27:
                return(1)
        if ea.easter(new_date.year).month == 4:
            if new_date.day in range(ea.easter(new_date.year).day-2,ea.easter(new_date.year).day+1):
                return(1)
    ## king/queen birthday
    if new_date.month == 6 & new_date.day > 7 & new_date.day < 15 & new_date.weekday() == 0:
        return(1)
    ##Labour day
    if new_date.month == 10 & new_date.day < 8 & new_date.weekday() == 0:
        return(1)
    ##christmas and boxing day
    if new_date.month == 12:
        if new_date.weekday() not in (5,6):
            if new_date.day in (25,26):
                return(1)
        elif new_date.weekday() == 5:
            if new_date.day in (27,28):
                return(1)
        else:
            if new_date.day in (26,27):
                return(1)
    return(0)
truedemand_nsw['public_holiday'] = truedemand_nsw['DATETIME'].apply(lambda x: public_holiday_check(x))

In [24]:
truedemand_nsw['public_holiday']

0          1
1          1
2          1
3          1
4          1
          ..
1323393    0
1323394    0
1323395    0
1323396    0
1323397    0
Name: public_holiday, Length: 1323398, dtype: int64

In [46]:
## temperature interpolation  to create temperatures that match the times of demands
## first model will be a basic linear combintion of nearest two surrounding existing temperatures
## will over fit and assumes strict linearity between temperatures which will be more inaccurate the larger the gaps 
## between surround temperatures
## the primary purpose of this is not to give the most accurate interpolation of temperature data but to allow
## us to create relatively reasonable temperature data to start the construction of models
## more sophisticated models will be used for final model construction

def basic_temp_interpolator(new_datetime_in: str):
    
    ##convert new_datetime into datetime data type from pandas
    ##input should be in the format "YYYY-mm-dd HH:MM:SS"
    new_datetime = pd.to_datetime(new_datetime_in)
    
    ##all data which occurs before new point
    previous_temp_data = temperature_nsw[temperature_nsw['DATETIME'] <= new_datetime]
    ##oldest point in that data, the point closest to the new data
    previous_temp_data = previous_temp_data[previous_temp_data['DATETIME'] == max(previous_temp_data['DATETIME'])]
    ##all data that occurs after new point
    next_temp_data = temperature_nsw[temperature_nsw['DATETIME'] >= new_datetime]
    ##newest point in that data, the point closest to the new data
    next_temp_data = next_temp_data[next_temp_data['DATETIME'] == min(next_temp_data['DATETIME'])]
    next_temp_data.reset_index()
    previous_temp_data.reset_index()
    
    ##avoids a divison by zero error in the case of interpolation calling on data already in temp data
    if previous_temp_data['DATETIME'].values[0] == new_datetime:
        return(previous_temp_data['TEMPERATURE'])
    
    ##time between exisitng measurements
    time_between_measurements = next_temp_data['DATETIME'].values[0] - previous_temp_data['DATETIME'].values[0]
    ##time between new point and closest measurement before it
    Previous_new_time_difference = new_datetime - previous_temp_data['DATETIME'].values[0]
    ##time between new point and closest measurement after it
    next_new_time_difference = time_between_measurements - Previous_new_time_difference
    
    ##this will be a measure of how close the new time is to the previous time 1 for equals 0 for equal to the next time
    proportion_previous = next_new_time_difference/time_between_measurements
    ##this will be the same as above but for the next time
    proportion_next = 1- proportion_previous
    
    new_temp = proportion_previous*previous_temp_data['TEMPERATURE'].values[0] + proportion_next*next_temp_data['TEMPERATURE'].values[0]

    return(new_temp)

##compiles without error but may need testing with data type conversion
##basic_temp_interpolator("2011-11-11 11:11:11")
##truedemand_nsw['TEMPERATURE'] = truedemand_nsw['DATETIME'].apply(basic_temp_interpolator)

In [16]:
## time series methods

## this will generate the average temperature for a defined time window either forwards or backwards
def prior_temp_average(time: str = '2012-01-01 00:00:00'
                       ,days: int = 0
                       ,hours: int = 1
                       ,minutes: int = 0
                       ,wheighted: bool = False
                       ,forward: bool = False):
    ##turn input str into a date time
    sample_datetime = pd.to_datetime(time)
    
    ##window of time before supplied date
    if forward:
        start_datetime = sample_datetime
        finish_datetime = sample_datetime + np.timedelta64(days,'D') + np.timedelta64(hours,'h') + np.timedelta64(minutes,'m')
    ##window of time after supplied date
    else:
        start_datetime = sample_datetime - np.timedelta64(days,'D') - np.timedelta64(hours,'h') - np.timedelta64(minutes,'m')
        finish_datetime = sample_datetime
    
    
    ##two methods wheighted = True will factor in the time around previous existing temperatures
    ##wheighted = False will return unweighted average
    if wheighted:
        ##not implemented yet
        return(None)
    else:
        temp_data = temperature_nsw[temperature_nsw['DATETIME'] <= finish_datetime]
        temp_data = temp_data[start_datetime <= temp_data['DATETIME']]
        return(temp_data.loc[:,'TEMPERATURE'].mean())
        